In [1]:
import pandas as pd

In [18]:
teams = {'Atlanta':'ATL', 'Boston':'BOS', 'Cleveland':'CLE', 'Charlotte':'CHO', 'Chicago':'CHI', 'Dallas':'DAL', 'Detroit':'DET', 'Denver':'DEN', 'Portland':'POR', 'Golden State':'GSW', 'Houston':'HOU',
         'Indiana':'IND', 'L.A. Clippers':'LAC', 'L.A. Lakers':'LAL', 'Memphis':'MEM', 'Miami':'MIA', 'Milwaukee':'MIL', 'Minnesota':'MIN', 'New Orleans':'NOP', 'New York':'NYK', 'Brooklyn':'BRK', 'Utah':'UTA',
         'Toronto':'TOR', 'Phoenix':'PHO', 'Philadelphia':'PHI', 'Washington':'WAS', 'San Antonio':'SAS', 'Orlando':'ORL', 'Oklahoma City':'OKC', 'Sacramento':'SAC'}

In [154]:
import os
frames = []
for season in os.listdir('odds/'):
    frames.append(pd.read_csv('odds/' + season + '/vegas.txt'))
df = pd.concat(frames)
df

,Date,Location,Team,OppTeam,TeamId,GameId,PercentBet_ML,Open_Line_ML,Pinnacle_ML,5dimes_ML,...,Average_Line_OU,Average_Odds_OU,Best_Line_OU,Worst_Line_OU,Best_Odds_OU,Worst_Odds_OU,Pts,Spread,Result,Total
0,2012-10-30,away,Washington,Cleveland,1610612764,21200001,44.6,220.0,210.0,200.0,...,190.33,-108.17,191.0,187.0,-104.0,-110.0,84.0,-10.0,L,178.0
1,2012-10-30,home,Cleveland,Washington,1610612739,21200001,55.4,-260.0,-235.0,-240.0,...,190.33,-110.17,191.0,187.0,-106.0,-115.0,94.0,10.0,W,178.0
2,2012-10-30,away,Boston,Miami,1610612738,21200002,44.8,220.0,244.0,240.0,...,186.58,-107.33,188.0,182.0,-104.0,-110.0,107.0,-13.0,L,227.0
3,2012-10-30,home,Miami,Boston,1610612748,21200002,55.2,-260.0,-275.0,-260.0,...,186.58,-109.33,188.0,182.0,-105.0,-115.0,120.0,13.0,W,227.0
4,2012-10-30,away,Dallas,L.A. Lakers,1610612742,21200003,40.8,350.0,393.0,420.0,...,186.00,-107.50,187.0,185.0,-100.0,-110.0,99.0,8.0,W,190.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447,2019-04-10,home,Denver,Minnesota,1610612743,21801228,45.2,-900.0,-794.0,-830.0,...,218.42,-107.17,218.5,218.0,-105.0,-110.0,99.0,4.0,W,194.0
2448,2019-04-10,away,Utah,L.A. Clippers,1610612762,21801229,61.6,199.0,249.0,275.0,...,225.50,-107.40,226.0,225.0,-102.0,-115.0,137.0,-6.0,L,280.0
2449,2019-04-10,home,L.A. Clippers,Utah,1610612746,21801229,38.4,-240.0,-281.0,-305.0,...,225.50,-106.60,226.0,225.0,-105.0,-110.0,143.0,6.0,W,280.0
2450,2019-04-10,away,Sacramento,Portland,1610612758,21801230,58.1,317.0,135.0,135.0,...,227.00,-107.67,229.0,226.5,-105.0,-111.0,131.0,-5.0,L,267.0


In [155]:
df = df.replace({'Team':teams, 'OppTeam':teams}).drop('TeamId', axis=1).drop('GameId', axis=1).drop('PercentBet_ML', axis=1)
def american_to_decimal(row):
    for col in df.columns:
        if 'ML' in col or 'Spread' in col:
            odds = float(row[col])
            if odds > 0: row[col] = 1/((odds/100)+1)
            elif odds < 0: row[col] = 1/((100/-odds)+1)
            elif odds == 0: row[col] = 0.5
            else: row[col] = -1
    return row[:13]
df = df.apply(american_to_decimal, axis=1)
df.to_csv('odds.csv')

In [156]:
bookmakers = dict()
for row in df.iloc:
    if row.Location == 'away':
        game = row.Date + row.Team + '-' + row.OppTeam
        bookmakers[game] = dict()
        for bookie in map(str, df.columns[4:]):
            if row[bookie] == -1: continue
            bookmakers[game][bookie[:-3]] = [row[bookie], 0]
    else:
        game = row.Date + row.OppTeam + '-' + row.Team
        for bookie in map(str, df.columns[4:]):
            if row[bookie] == -1: continue
            bookmakers[game][bookie[:-3]][1] = row[bookie]

In [157]:
import json
json.dump(bookmakers, open('odds.json', 'w'))